In [3]:
import pandas as pd 
import numpy as np
import csv
import os.path

In [6]:
#Importing the data from the csv file in the Data folder and turn them into pandas dataframe. Some of them needs to be converted to int datatype to combina all the datasets because some of the dataframe is type object when read'''
champions = pd.read_csv(r'./Data/champs.csv')
matches = pd.read_csv(r'./Data/matches.csv', dtype = np.array(np.dtype('int16')))
stats1 = pd.read_csv(r'./Data/stats1.csv', dtype = np.array(np.dtype('int16')))
stats1_2 = pd.read_csv(r'./Data/stats1-2.csv', dtype = np.array(np.dtype('int16')))
stats1_3 = pd.read_csv(r'./Data/stats1-3.csv', dtype = np.array(np.dtype('int16')))
stats2 = pd.read_csv(r'./Data/stats2.csv', dtype = np.array(np.dtype('int16')))
stats2_2 = pd.read_csv(r'./Data/stats2-2.csv', dtype = np.array(np.dtype('int16')))

#Appending stats1 and stats2 dataset
stats1_2 = pd.concat([stats1_2,stats1_3],sort = False)#stats1_2.concat(stats1_3,, sort = False);
stats1 = pd.concat([stats1,stats1_2],sort = False)#stats1.concat(stats1_2);
stats2 = pd.concat([stats2,stats2_2],sort = False)#stats2.concat(stats2_2);
stats =pd.concat([stats1,stats2],sort = False)# stats1.concat(stats2)



In [8]:
#Merge particpants with appropriate champion name. test
participants = pd.read_csv(r'./Data/participants.csv', dtype = np.array(np.dtype('int16')))
#Rename the column of the participants dataframe from id to play_id and championid to id for better clarification. inplace so no extra copy is made
participants.rename(columns ={'id': 'player_id','championid':'id'}, inplace = True)
#Merge the participants dataset with the champions dataset based on the championid. This is similar to left join in SQL 
champions['id'] = champions['id'].astype(str).astype(int)
participants['id'] = participants['id'].astype(str).astype(int)
participants = pd.concat([participants.set_index('id'),champions.set_index('id')], axis=1, join='inner').reset_index()
#Delete the column role because I think it's irrelevant for now 

#Rename the stats data set column from id to player_id and inplace 
stats.rename(columns={'id':'player_id'},inplace = True)
#Convert the player_id column to int because it is currently type Object and we cannot manipulate object type column and int type column too easily 
stats['player_id'] = stats['player_id'].astype(int)
participants['player_id'] = participants['player_id'].astype(str).astype(int)
#merge the participants dataset with stats dataset based on the player_id. This makes participants a massive dataset with lots of columns and champions that each player uses
participants = participants.merge(stats, how = 'left', on = 'player_id')
#This is me testing out the gorupby but this will group the champion based on their name and count how many win and lose they have. Win and lose are represented with 0 and 1 
champsFreq = participants.groupby(['name','win'])['win'].count()

#Turn it into a dataframe
champsFreq = pd.DataFrame(champsFreq)
#Add Suffix Count to the second column. When we groupby, it creates columns with same name and it's difficult to select columns so I add a letter to the column with the number of wins and lose for easy access to the dataframe
champsFreq = champsFreq.add_suffix('Count').reset_index()
#Converting the columns from object to int type
champsFreq['win'] = champsFreq['win'].astype(str).astype(int)

In [79]:
#Get the lose values
loseValues = champsFreq.loc[champsFreq['win'] == 0,'winCount'].reset_index()
loseValues.rename(columns = {'winCount': 'lose'}, inplace = True)
loseValues = loseValues['lose']

#Get the win values
winValues = champsFreq.loc[champsFreq['win'] == 1, 'winCount'].reset_index()
winValues.rename(columns = {'winCount':'win'}, inplace = True)
winValues = winValues['win']

#Get the champion name
championName = champsFreq['name'].drop_duplicates().reset_index()
championName = championName['name']

In [10]:
#get the match data for season 8 only
matches['seasonid'] = matches['seasonid'].astype(int)
matches = matches[matches['seasonid'] == 8]

In [21]:
#get the match id specific columns good for evaluation 
matches.rename(columns = {'id': 'matchid'}, inplace = True)
matches = matches[['matchid','seasonid','duration','version']]

In [24]:
#get participants columns that needed 
participants_match = participants[['id','player_id','matchid','player','name','win']]

In [27]:
#merge on matchid 
new_match = matches.merge(participants_match, how = 'left', on = 'matchid')

In [81]:
#Remake the chamions dataframe with win and lose values for each champion to make a clearer dataframe
champions_values = pd.DataFrame([championName, winValues, loseValues]).transpose()

In [85]:
new_match

,matchid,seasonid,duration,version,id,player_id,player,name,win
0,10,8,1909,7.10.187.9675,19,9,1,Warwick,0
1,10,8,1909,7.10.187.9675,267,10,2,Nami,0
2,10,8,1909,7.10.187.9675,119,11,3,Draven,0
3,10,8,1909,7.10.187.9675,114,12,4,Fiora,0
4,10,8,1909,7.10.187.9675,112,13,5,Viktor,0
5,10,8,1909,7.10.187.9675,72,14,6,Skarner,1
6,10,8,1909,7.10.187.9675,3,15,7,Galio,1
7,10,8,1909,7.10.187.9675,103,16,8,Ahri,1
8,10,8,1909,7.10.187.9675,222,17,9,Jinx,1
9,10,8,1909,7.10.187.9675,161,18,10,VelKoz,1
